In [1]:
from WindowCapture import WindowCapture

import torch
from torch.autograd import Variable

from os.path import join
from glob import glob
import numpy as np
import skimage.io as io
from skimage.transform import resize

from C3D_model import C3D

from PIL import Image


In [2]:

def read_labels_from_file(filepath):
    """
    Reads Sport1M labels from file
    
    Parameters
    ----------
    filepath: str
        the file.
        
    Returns
    -------
    list
        list of sport names.
    """
    with open(filepath, 'r',encoding="utf8") as f:
        labels = [line.strip() for line in f.readlines()]
    return labels
   


In [3]:
net = C3D()
print(net)

net.load_state_dict(torch.load('C3D_Accu_90.pt'))
net.cuda()
net.eval()

def model_1(X,nt):
    X = Variable(X)
    X = X.cuda()
    prediction = nt(X)
    prediction = prediction.data.cpu().numpy()
   
    # read labels
    labels = read_labels_from_file('labels1.txt')

    # print top predictions
    top_inds = prediction[0].argsort()[::-1][:5]  # reverse sort and take five largest items
#     print('\nTop 5:')
#     for i in top_inds:
#         print('{:.5f} {}'.format(prediction[0][i], labels[i]))
    print('{:.1f} {}'.format(prediction[0][top_inds[0]], labels[top_inds[0]]))
    

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [5]:

wincap = WindowCapture('cornor.mkv')
i=1
img=[]
while(True):
    screenshot = wincap.get_screenshot()
    
    img.append(screenshot)
    if len(img)==16:
        clip = np.array([resize(frame, output_shape=(112, 200), preserve_range=True) for frame in img])
#         print(clip.shape)
        img=[]
        clip = clip[:, :, 44:44+112, :]  # crop centrally
        clip = clip.transpose(3, 0, 1, 2)  # ch, fr, h, w
        clip = np.expand_dims(clip, axis=0)  # batch axis
        clip = np.float32(clip)      
        model_1(torch.from_numpy(clip),net)
        
        

0.5 No Fights
0.5 No Fights
0.5 No Fights
0.5 No Fights
0.5 No Fights
0.5 No Fights


KeyboardInterrupt: 